<a href="https://colab.research.google.com/github/blakelobato/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module2-sql-for-analysis/titanic_insert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import os
from dotenv import load_dotenv
import pandas as pd
import sqlite3
import psycopg2
from psycopg2.extras import execute_values

load_dotenv() # looks inside the .env file for some env vars

# passes env var values to python var
DB_HOST = os.getenv("DB_HOST", default="OOPS")
DB_NAME = os.getenv("DB_NAME", default="OOPS")
DB_USER = os.getenv("DB_USER", default="OOPS")
DB_PASSWORD = os.getenv("DB_PASSWORD", default="OOPS")

DB_FILEPATH = os.path.join(os.path.dirname(__file__), "titanic.csv")

class titanic_insert():
    def __init__(self):
        self.sqlite_connection = sqlite3.connect(DB_FILEPATH)
        self.sqlite_cursor = self.sqlite_connection.cursor()
        self.pg_connection = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
        self.pg_cursor = self.pg_connection.cursor()

    def create_people_class_querry(self):
        create_query = """
        DROP TABLE IF EXISTS people_class; -- allows this to be run idempotently, avoids psycopg2.errors.UniqueViolation: duplicate key value violates unique constraint "characters_pkey" DETAIL:  Key (character_id)=(1) already exists.
        CREATE TABLE IF NOT EXISTS people_class (
            id SERIAL PRIMARY KEY,
            name VARCHAR(30),
            pclass INT,
            gender VARCHAR(30),
            lived BOOL,
            age INT,
            fare FLOAT
        );
        """
        print(create_query)
        self.pg_cursor.execute(create_query)
        self.pg_connection.commit()

    def insert_people(self, people):
        insertion_query = "INSERT INTO characters (id, name, pclass, gender, lived, age, fare) VALUES %s"
        list_of_tuples = people
        execute_values(self.pg_cursor, insertion_query, list_of_tuples)
        self.pg_connection.commit()

if __name__ == "__main__":

    service = titanic_insert()

    #
    # EXTRACT AND TRANSFORM
    #

    people = service.get_people()
    print(type(people), len(people))
    print(people[0])

    #
    # LOAD
    #

    service.create_people_table()

    service.insert_people(people)